In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote
from PIL import Image, ImageFile
from openpyxl import Workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash, sanitize_filename

In [3]:
def getsizes(uri, headers):
    # get file size *and* image size (None if not known)
    image_url = quote(image.encode('utf-8'), ':/')
    try: 
        r = requests.get(image_url, headers=headers)
        file = Image.open(BytesIO(r.content))
    except :
        return 0, None

    size = r.headers["content-length"]
    if size:
        size = int(size)

    p = ImageFile.Parser()
    while True:
        data = file.read(1024)
        if not data:
            break
        p.feed(data)
        if p.image:
            return size, p.image.size
            break
    file.close()
    return size, None

In [4]:
commons_site = pb.Site("commons", "commons")
wb = Workbook()
ws = wb.active
EXCEL_FILE = "diario.madrid.es.xlsx"

In [13]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [6]:
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
headers = {'User-Agent': user_agent}
url_base = 'https://diario.madrid.es/blog/notas-de-prensa/'

r = requests.get(url_base, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
pages = soup.find_all("a", class_="page-numbers")
final_page = int(pages[-2].get_text())-1
final_page

202

In [20]:
page_number = 2
row = []
for i in range (final_page) :
    if i > page_number :
        break

    if i == 0 :
        url = url_base
    else :
        url = '{0}page/{1}'.format(url_base, i+1)
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    articles = soup.find_all("h2", class_="post-title")
    for article in articles :
        url = article.a["href"]
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
        date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
        year = date.split('-')[0]
        month = calendar.month_name[int(date.split('-')[1])]
        body = soup.find_all("div", class_="post-content")
        p_description = body[0].find_all("p")
        #print (p_description)
        for p in p_description :
            if len(p.get_text()) > 10:
                description = p.get_text()
                break

        images = [image.a["href"] for image in soup.find_all("div", class_="gallery-icon")]
        counter = 0
        for image in images:
            try:
                href = quote(image.encode('utf-8'), ':/')
            except :
                continue
            print (href)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)
            print (image_path)
            try:
                r = requests.get(href, headers=headers)
                image = Image.open(BytesIO(r.content))
                #image.save(image_path)
                row = [image_path, href, title, url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

        images = soup.find_all(attrs={'class': re.compile(r"wp-image-\d*")})
        for i, image in enumerate(images) :
            try:
                links = {int(x.strip().split(' ')[1][:-1]): x.strip().split(' ')[0] for x in image.get('srcset').split(',')}
                sortedlist = [(k, links[k]) for k in sorted(links)]
                link = sortedlist[-1][1]
            except :
                link = image.get('src')
            if image.parent.get('href') == None :
                image_url = sortedlist[-1][1]
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            elif getsizes(image.parent.get('href'), headers)[0] > getsizes(link, headers)[0] :
                image_url = image.parent.get('href')
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            else :
                image_url = sortedlist[-1][1]
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)

            print (image_path)
            try:
                r = requests.get(href, headers=headers)
                image = Image.open(BytesIO(r.content))
                #image.save(image_path)
                row = [image_path, href, title, url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

https://diario.madrid.es/wp-content/uploads/2018/01/IMG_20180123_180325-1333x1000.jpg
/home/paws/commons-helpers/images/Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 01.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/IMG_20180123_175422-1333x1000.jpg
/home/paws/commons-helpers/images/Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 02.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/IMG_20180123_175845-1333x1000.jpg
/home/paws/commons-helpers/images/Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 03.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/DSC_4818-768x1156.jpg
/home/paws/commons-helpers/images/Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 01.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/DSC_4818-768x1156.jpg
/home/paws/commons-helpers/images/Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Cañ

https://diario.madrid.es/wp-content/uploads/2018/01/image1-2.jpeg
/home/paws/commons-helpers/images/Bomberos Madrid participa en el ‘V Workshop de emergencias por riesgos tecnológicos’ 04.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/0001-768x1087.jpg
/home/paws/commons-helpers/images/‘Cursos infanto juveniles’ en los centros culturales de Moratalaz 01.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/20180121144128_097A1983-1500x1000.jpg
/home/paws/commons-helpers/images/Marcelino tendrá su calle y su estatua en Madrid 01.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/20180121130326_097A1805-1500x1000.jpg
/home/paws/commons-helpers/images/Marcelino tendrá su calle y su estatua en Madrid 02.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/20180121130654_097A1876-1500x1000.jpg
/home/paws/commons-helpers/images/Marcelino tendrá su calle y su estatua en Madrid 03.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/20180121131219_097A1882-1500x1000.jpg
/

https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaPalacioCongresos_4-1500x1000.jpg
/home/paws/commons-helpers/images/Carmena - “La unidad de las instituciones permite relanzar el Palacio de Congresos de la Castellana” 02.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaPalacioCongresos_3-1500x1000.jpg
/home/paws/commons-helpers/images/Carmena - “La unidad de las instituciones permite relanzar el Palacio de Congresos de la Castellana” 03.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaPalacioCongresos_1-1408x1000.jpg
/home/paws/commons-helpers/images/Carmena - “La unidad de las instituciones permite relanzar el Palacio de Congresos de la Castellana” 04.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/1-Martirio-en-el-2018-699x1000.jpg
/home/paws/commons-helpers/images/Conde Duque acoge la muestra de fotos ‘La generación del 87. Orígenes y destinos, 1987 2017’ 01.jpg
https://diario.madrid.es/wp-content/uploads/

In [ ]:
wb.save(os.path.join(cwd, EXCEL_FILE))